## Keypoint Detection:
This notebook detects facial keypoints via webcam input.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2 as cv
%matplotlib inline

In [ ]:
import torch
#from models import Net
from models import mod_cnn

#net = Net()
net =mod_cnn()

## TODO: load the best saved model parameters (by your path name)
## You'll need to un-comment the line below and add the correct name for *your* saved model
#net.load_state_dict(torch.load('saved_models/keypoints_model_1.pt'))

net.load_state_dict(torch.load('saved_models/keypoints_model_with5x5_1x1_final.pt'))

## print out your net and prepare it for testing (uncomment the line below)
net.eval()

In [ ]:
def draw_moustache(img,key_pts):
    image_copy = np.copy(img)

    # top-left location for sunglasses to go
    # 3 = edge of left eyebrow
    x = int(key_pts[3, 0])
    y = int(key_pts[3, 1])

    # height and width of sunglasses
    # h = length of nose
    h = int(abs(key_pts[62,1] - key_pts[57,1]))
    # w = left to right eyebrow edges
    w = int(abs(key_pts[3,0] - key_pts[13,0]))

    # read in sunglasses
    sunglasses = cv.imread('images/moustache.png', cv.IMREAD_UNCHANGED)
    # resize sunglasses
    new_sunglasses =  cv.resize(sunglasses, (w, h), interpolation = cv.INTER_CUBIC)

    # get region of interest on the face to change
    roi_color = image_copy[y:y+h,x:x+w]

    # find all non-transparent pts
    ind = np.argwhere(new_sunglasses[:,:,3] > 0)

    # for each non-transparent point, replace the original image pixel with that of the new_sunglasses
    for i in range(3):
        roi_color[ind[:,0],ind[:,1],i] = new_sunglasses[ind[:,0],ind[:,1],i]    
    # set the area of the image to the changed region with sunglasses
    image_copy[y:y+h,x:x+w] = roi_color
    
    return image_copy

In [ ]:
def draw_sunglasses(img,key_pts):
    image_copy = np.copy(img)

    # top-left location for sunglasses to go
    # 18 = edge of left eyebrow
    x = int(key_pts[17, 0])
    y = int(key_pts[17, 1])

    # height and width of sunglasses
    # h = length of nose
    h = int(abs(key_pts[27,1] - key_pts[34,1]))
    # w = left to right eyebrow edges
    w = int(abs(key_pts[17,0] - key_pts[26,0]))

    # read in sunglasses
    sunglasses = cv.imread('images/sunglasses.png', cv.IMREAD_UNCHANGED)
    # resize sunglasses
    new_sunglasses =  cv.resize(sunglasses, (w, h), interpolation = cv.INTER_CUBIC)

    # get region of interest on the face to change
    roi_color = image_copy[y:y+h,x:x+w]

    # find all non-transparent pts
    ind = np.argwhere(new_sunglasses[:,:,3] > 0)

    # for each non-transparent point, replace the original image pixel with that of the new_sunglasses
    for i in range(3):
        roi_color[ind[:,0],ind[:,1],i] = new_sunglasses[ind[:,0],ind[:,1],i]    
    # set the area of the image to the changed region with sunglasses
    image_copy[y:y+h,x:x+w] = roi_color
    
    return image_copy

In [ ]:
capture=cv.VideoCapture(0)

face_detector=cv.CascadeClassifier('Haarcascades/haarcascade_frontalface_default.xml')

def drawKeyPts(im,keyp,col,th):
    for curKey in keyp:
        x=np.int(curKey[0])
        y=np.int(curKey[1])
        size = np.int(curKey.size)
        cv.circle(im,(x,y),size, col,thickness=th, lineType=8, shift=0) 
    #plt.imshow(im)    
    return im   

while (True):
    ret, img=capture.read()
    #Haar Cascade Classifier
    faces=face_detector.detectMultiScale(img,1.3,5)
    
    padding=100
    for (x,y,w,h) in faces:
        #if w >100:
            #Draw bounding rectangle.
            cv.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
            #Crop the image.
            face_detect=img[int(y)-padding:int(y+h)+padding,int(x)-padding:int(x+w)+padding]
            #Resize to VGGFace dimensions.
            face_detect=cv.resize(face_detect,(224,224))
            #face_detect_rgb=face_detect.copy()
            #PreProcess image.
            face_detect_gray=cv.cvtColor(face_detect,cv.COLOR_BGR2GRAY)
            face_detect_gray=face_detect_gray/255.0

            roi=torch.from_numpy(face_detect_gray)
            roi.resize_(1,1,224,224)
            #roi.unsqueeze_(0)
            roi = roi.type(torch.FloatTensor)
           #print('torch shape',roi.shape)

            ## TODO: Make facial keypoint predictions using your loaded, trained network 
            output=net(roi)

            ## TODO: Display each detected face and the corresponding keypoints        
            output_pts = output.view(output.size()[0], 68, -1)  

            predicted_key_pts = output_pts[0].data
            predicted_key_pts = predicted_key_pts.numpy()
            # undo normalization of keypoints  
            predicted_key_pts = predicted_key_pts*50.0+100
            zero_out=np.zeros((1,1))
            #print(type(predicted_key_pts),predicted_key_pts.shape)
            outimg_keypts = drawKeyPts(img,predicted_key_pts,(0,255,0),2)
            outimg=draw_sunglasses(face_detect,predicted_key_pts)
            #outimg=draw_moustache(outimg,predicted_key_pts)
            # blob=cv.drawKeypoints(img,predicted_key_pts,zero_out,(255,0,255),cv.DRAW_MATCHES_FLAGS_DEFAULT)

            cv.imshow('Image',outimg)
            cv.imshow('Orig',face_detect)
            cv.imshow('keypts',outimg_keypts)
            if cv.waitKey(1) == 13: #13 is the Enter Key
                break
capture.release()
cv.destroyAllWindows()
        